In [1]:
import torch
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch.nn as nn


In [2]:
#device configuration
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
#Hyperparameters
num_epochs=4
batch_size=4
learning_rate=0.001

In [7]:
#dataset has PILImage images of range [0,1].
#We transform them to Tensors of normalized range[-1,1]
data_transformer = transforms.Compose([
        # transforms.RandomResizedCrop((150,150)),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
        ])

In [8]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=data_transformer)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=data_transformer)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
train_loader =  torch.utils.data.DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader =  torch.utils.data.DataLoader(test_dataset,batch_size=64,shuffle=True)
classes=('plane','car','bird','cat','deer','dog','frog','hourse','ship','truck')